# **Training SSD with MobileNet V3 Large**

This notebook provides a step-by-step guide to train the SSD model with a MobileNet V3 Large 320 FPN backbone, specifically designed for vehicle detection tasks.

**NOTE:** This notebook only guides you for detecting a single class, so adjust code according to your needs. Annd adjust parameters like number of epochs, learning rate(**lr**), weight decay, momentum as per your requirements.

### Prerequisites

Ensure you have the following libraries installed:
- PyTorch
- torchvision
- PIL
- pycocotools
- numpy
- tqdm
- json

In [ ]:
!pip install torch torchvision
!pip install pillow
!pip install numpy
!pip install tqdm
!pip install pycocotools
!pip install tensorboard

## Step 1: Dataset Preparation

Refer to the following video for guidelines on how to obtain, label (in COCO format), and use a dataset for SSD model training: [COCO Dataset Preparation Video](https://youtu.be/O-ZPxTpb2Yg?feature=shared)

### Cell for Downloading Dataset

Use the cell below to paste the code for downloading your dataset from Roboflow or another source.

## Step 2: Enable GPU on Google Colab

If you're using Google Colab, follow this guide to enable GPU acceleration: [Enabling GPU in Google Colab](https://www.geeksforgeeks.org/how-to-use-gpu-in-google-colab/)

## Step 3: Import Libraries
This step is essential to import necessary libraries.

In [ ]:
import os
import torch
from torchvision.transforms import functional as F
from PIL import Image, ImageDraw
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import ssdlite320_mobilenet_v3_large
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from tqdm import tqdm
import numpy as np
import json

## Step 4: Define Custom Dataset Class

Modify the file paths for your dataset and annotations here.

In [ ]:
class VehiclesDataset(Dataset):
    def __init__(self, annotation_file, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.coco = COCO(annotation_file)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        coco = self.coco
        img_id = self.ids[idx]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        coco_annotation = coco.loadAnns(ann_ids)
        path = coco.loadImgs(img_id)[0]['file_name']

        img = Image.open(os.path.join(self.root_dir, path)).convert('RGB')

        num_objs = len(coco_annotation)
        boxes = []
        labels = []

        for i in range(num_objs):
            if coco_annotation[i]['category_id'] == 1:  # Assuming '1' is for 'Vehicles'
                xmin = coco_annotation[i]['bbox'][0]
                ymin = coco_annotation[i]['bbox'][1]
                xmax = xmin + coco_annotation[i]['bbox'][2]
                ymax = ymin + coco_annotation[i]['bbox'][3]
                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(coco_annotation[i]['category_id'])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([img_id])}

        if self.transform:
            img = self.transform(img)

        return img, target

def collate_fn(batch):
  batch = [data for data in batch if data is not None and data[0] is not None]
  return tuple(zip(*batch))

transform = transforms.Compose([transforms.ToTensor()])

# Replace with your dataset paths
train_dataset = VehiclesDataset(r'<your_train_annotations.json>', r'<your_train_dataset_path>', transform)
val_dataset = VehiclesDataset(r'<your_val_annotations.json>', r'<your_val_dataset_path>', transform)

## Step 5: Data Loaders

Set up data loaders for training and validation datasets. Play around in choosing number of batches to see what suits you best.bold text

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

## Step 6: Initialize the Model


### For First-Time Training

In [ ]:
model = ssdlite320_mobilenet_v3_large(pretrained=True)

### For Training from a Checkpoint

In [ ]:
# Create the model from the pretrained model
model = ssdlite320_mobilenet_v3_large(pretrained=True)

# Load the pretrained weights
model.load_state_dict(torch.load(r'<your_model_name>.pth'))

## Step 7: Set Up Training Utilities

Configure the device, optimizer, learning rate scheduler, and TensorBoard writer.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3)
writer = SummaryWriter()

## Step 8: Training Loop

This cell will train the model and save checkpoints after each epoch.

In [ ]:
def evaluate(model, data_loader, device, coco_gt):
    model.eval()
    results = []

    with torch.no_grad():
        for images, targets in data_loader:
            images = list(img.to(device) for img in images)
            outputs = model(images)

            for i, output in enumerate(outputs):
                image_id = targets[i]['image_id'].item()
                for box, score, label in zip(output['boxes'], output['scores'], output['labels']):
                    box = box.to('cpu').numpy()
                    score = score.to('cpu').numpy()
                    label = label.to('cpu').numpy()

                    result = {
                        "image_id": image_id,
                        "category_id": int(label),
                        "bbox": [box[0], box[1], box[2] - box[0], box[3] - box[1]],  # Convert to x,y,w,h format
                        "score": score
                    }
                    results.append(result)

    if results:
        coco_dt = coco_gt.loadRes(results)  # Load predictions
        coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()
        mAP = coco_eval.stats[0]  # Average Precision IoU=0.50:0.95
    else:
        mAP = 0

    return mAP

In [ ]:
def save_checkpoint(epoch, model, optimizer, lr_scheduler, save_path):
    """Saves a checkpoint of the current model state."""
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'lr_scheduler_state_dict': lr_scheduler.state_dict() if lr_scheduler else None,
    }, save_path)

num_epochs = 300
checkpoint_path = 'ssd_checkpoint.pth'

val_annotation_path = r'<your_val_annotations.json>'
val_coco_gt = COCO(val_annotation_path)

try:
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, targets in tqdm(train_loader):
            # Filter out images without any annotations
            valid_images, valid_targets = zip(*[(img, target) for img, target in zip(images, targets) if target['boxes'].size(0) != 0])

            # Skip batch if no valid images are found
            if not valid_images:
                continue

            valid_images = list(img.to(device) for img in valid_images)
            valid_targets = [{k: v.to(device) for k, v in t.items()} for t in valid_targets]

            optimizer.zero_grad()

            loss_dict = model(valid_images, valid_targets)
            losses = sum(loss for loss in loss_dict.values())

            if losses.item() != 0:
                losses.backward()
                optimizer.step()

            running_loss += losses.item()

        # Calculate training loss for the epoch
        train_loss = running_loss / len(train_loader)
        writer.add_scalar('Training loss', train_loss, epoch)

        # Validation step
        mAP = evaluate(model, val_loader, device, val_coco_gt)
        writer.add_scalar('Validation Loss', epoch)
        writer.add_scalar('Validation mAP', mAP, epoch)

        # Step the learning rate scheduler with the validation mAP
        lr_scheduler.step(mAP)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation mAP: {mAP:.4f}")

        # Save checkpoint at the end of the epoch
        save_checkpoint(epoch, model, optimizer, lr_scheduler, checkpoint_path)
        print(f"Checkpoint saved for epoch {epoch + 1}")

except KeyboardInterrupt:
    print("Training interrupted, saving current model state...")
    save_checkpoint(epoch, model, optimizer, lr_scheduler, checkpoint_path)
    print("Checkpoint saved.")

writer.close()

## Step 9: Save and Load Model

### Save Model

In [ ]:
torch.save(model.state_dict(), '<your_model_name>.pth')

### Load Model

In [ ]:
model.load_state_dict(torch.load('<your_model_path>.pth'))

## Step 10: Evaluation and Inference

### Model Evaluation

In [ ]:
# Load the model
model = ssdlite320_mobilenet_v3_large(pretrained=True)

model.load_state_dict(torch.load(r'<your_model_path>.pth'))
model.to('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()

# Load the validation dataset
val_annotation_file = r'<your_val_annotations.json>'
coco = COCO(val_annotation_file)
img_ids = coco.getImgIds()

# Function to convert image id to file path
def get_image_path(coco, img_id):
    img_info = coco.loadImgs(img_id)[0]
    return r'<your_val_dataset_path>\\' + img_info['file_name']

# Running inference and preparing results for COCO format
results = []
for img_id in img_ids:
    img_path = get_image_path(coco, img_id)
    img = Image.open(img_path).convert('RGB')
    img_tensor = F.to_tensor(img).unsqueeze(0).to('cuda' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():
        preds = model(img_tensor)[0]

    for box, label, score in zip(preds['boxes'], preds['labels'], preds['scores']):
        box = box.cpu().numpy()
        bbox = [float(box[0]), float(box[1]), float(box[2] - box[0]), float(box[3] - box[1])]  # Convert to Python floats
        results.append({
            'image_id': img_id,
            'category_id': int(label.cpu().item()),  # Convert to int
            'bbox': bbox,
            'score': float(score.cpu().item())  # Convert to Python float
        })

# Write results to a file
with open('predictions.json', 'w') as f:
    json.dump(results, f)

# Load the predictions with COCO API
coco_pred = coco.loadRes('predictions.json')

# Running the COCO evaluation
coco_eval = COCOeval(coco, coco_pred, 'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

### Inference and Visualization

In [ ]:
model.load_state_dict(torch.load(r'your_model_path>.pth'))
model.eval()
model.to(device)

# Function to apply the model to an image and draw the results
from IPython.display import display

def infer_and_draw(image_path, model, device, threshold=0.9):
    img = Image.open(image_path).convert("RGB")
    img_tensor = F.to_tensor(img).to(device)

    with torch.no_grad():
        prediction = model([img_tensor])

    draw = ImageDraw.Draw(img)
    found_boxes = False

    for element in range(len(prediction[0]['boxes'])):
        score = prediction[0]['scores'][element]
        if score > threshold:
            found_boxes = True
            box = prediction[0]['boxes'][element].cpu().numpy()
            label = prediction[0]['labels'][element].cpu().numpy()
            label_name = 'Vehicles' if label == 1 else 'Background'

            draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline ="red", width=3)
            draw.text((box[0], box[1]), f"{label_name}: {score:.2f}", fill="red")

    if found_boxes:
        print("Bounding boxes found and drawn.")
    else:
        print("No bounding boxes found above the threshold.")

    return img

# Run inference on an image
image_path = r'your_image_path>.jpg'
result_img = infer_and_draw(image_path, model, device)
display(result_img)  # Use this in Jupyter Notebook
#result_img.show()  # Use this in a Python script

**Conclusion**

This notebook is designed to be a user-friendly guide for beginners. By following these steps, you can train a SSD model for vehicle detection or other similar tasks, customize file paths, checkpoint names, and model names as per your requirements.